# XGBOOST model baseline - 1 year
- run model from 1_year_combined data with feature engineering
  - TAIL_NUM causes OOM error, comment out for now
- featuring engineering handled in https://dbc-fae72cab-cf59.cloud.databricks.com/editor/notebooks/1792055957780055?o=4021782157704243



## Imports

In [0]:
from pyspark.sql.functions import col
from pyspark.sql import Window
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator


import random

import mlflow
print(mlflow.__version__)

import os

spark.conf.set("spark.databricks.mlflow.trackMLlib.enabled", "true")

RANDOM_SEED = 0
# Define experiment name with proper Databricks path
EXPERIMENT_NAME = "/Shared/team_2_2/mlflow-baseline"
# Create the experiment if it doesn't exist
try:
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if experiment is None:
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
        print(f"Created new experiment with ID: {experiment_id}")
    else:
        print(f"Using existing experiment: {experiment.name}")
    mlflow.set_experiment(EXPERIMENT_NAME)
except Exception as e:
    print(f"Error with experiment setup: {e}")
    # Fallback to default experiment in workspace
    mlflow.set_experiment(f"/Users/{dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()}/default")



## Helper Functions


In [0]:
def checkpoint_dataset(dataset, file_path):
    # Create base folder
    section = "2"
    number = "2"
    base_folder = f"dbfs:/student-groups/Group_{section}_{number}"
    dbutils.fs.mkdirs(base_folder)
    # Create subfolders if file_path contains directories
    full_path = f"{base_folder}/{file_path}.parquet"
    subfolder = "/".join(full_path.split("/")[:-1])
    dbutils.fs.mkdirs(subfolder)
    # Save dataset as a parquet file
    dataset.write.mode("overwrite").parquet(full_path)
    print(f"Checkpointed {file_path}")

## Datasets - custom join

In [0]:
%fs ls dbfs:/student-groups/Group_2_2/1_year_custom_joined/feature_eng

In [0]:
checkpoint_path = f"dbfs:/student-groups/Group_2_2"
# dataset_path = f"{checkpoint_path}/1_year_custom_joined/raw_data/training_splits"
dataset_path = f"{checkpoint_path}/1_year_custom_joined/feature_eng/training_splits"

# Read datasets from checkpoint
train_df = spark.read.parquet(f"{dataset_path}/train.parquet")
validation_df = spark.read.parquet(f"{dataset_path}/validation.parquet")

# Feature Selection

In [0]:
baselines_columns = [
    "QUARTER",
    "MONTH",
    "YEAR",
    "DAY_OF_MONTH",
    "DAY_OF_WEEK",
    "OP_CARRIER",
    # "TAIL_NUM",
    "ORIGIN_AIRPORT_SEQ_ID",
    "DEST_AIRPORT_SEQ_ID",
    "CRS_ELAPSED_TIME",
    "DISTANCE",
    "DEP_DELAY_NEW",
    "utc_timestamp",
    "CRS_DEP_MINUTES",            # feature eng start
    "prev_flight_delay_in_minutes", 
    "prev_flight_delay",
    "origin_delays_4h",
    "delay_origin_7d",
    "delay_origin_carrier_7d",
    "delay_route_7d",
    "flight_count_24h",
    "LANDING_TIME_DIFF_MINUTES",
    "AVG_ARR_DELAY_ORIGIN",
    "AVG_TAXI_OUT_ORIGIN",        # feature eng end
    'HourlyDryBulbTemperature',     # weather start
    'HourlyDewPointTemperature',
    'HourlyRelativeHumidity',
    'HourlyAltimeterSetting',
    'HourlyVisibility',
    'HourlyStationPressure',
    'HourlyWetBulbTemperature',
    'HourlyPrecipitation',
    'HourlyCloudCoverage',
    'HourlyCloudElevation',
    'HourlyWindSpeed'               # weather end
]

In [0]:
train_df = train_df.filter(F.col("DEP_DELAY_NEW").isNotNull()).select(baselines_columns)
validation_df = validation_df.filter(F.col("DEP_DELAY_NEW").isNotNull()).select(baselines_columns)

In [0]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor

# Categorical encoding
carrier_indexer = StringIndexer(inputCol="OP_CARRIER", outputCol="carrier_idx", handleInvalid="keep")
origin_indexer = StringIndexer(inputCol="ORIGIN_AIRPORT_SEQ_ID", outputCol="origin_idx", handleInvalid="keep")
dest_indexer = StringIndexer(inputCol="DEST_AIRPORT_SEQ_ID", outputCol="dest_idx", handleInvalid="keep")
tail_num_indexer = StringIndexer(inputCol="TAIL_NUM", outputCol="tail_num_idx", handleInvalid="keep")

carrier_encoder = OneHotEncoder(inputCol="carrier_idx", outputCol="carrier_vec")
origin_encoder = OneHotEncoder(inputCol="origin_idx", outputCol="origin_vec")
dest_encoder = OneHotEncoder(inputCol="dest_idx", outputCol="dest_vec")
tail_num_encoder = OneHotEncoder(inputCol="tail_num_idx", outputCol="tail_num_vec")



In [0]:
# Assemble all features
assembler = VectorAssembler(
    inputCols=[
        "QUARTER",
        "MONTH", 
        "YEAR",
        "DAY_OF_MONTH",
        "DAY_OF_WEEK",
        "carrier_vec",
        "origin_vec",
        "dest_vec",
        "CRS_ELAPSED_TIME",
        "DISTANCE",
        # "tail_num_vec",
        "CRS_DEP_MINUTES",                 # feature eng start
        "prev_flight_delay_in_minutes",
        "prev_flight_delay",
        "origin_delays_4h",
        "delay_origin_7d",
        "delay_origin_carrier_7d",
        "delay_route_7d",
        "flight_count_24h",
        "LANDING_TIME_DIFF_MINUTES",
        "AVG_ARR_DELAY_ORIGIN",
        "AVG_TAXI_OUT_ORIGIN",              # feature eng end
        'HourlyDryBulbTemperature',         # weather start
        'HourlyDewPointTemperature',
        'HourlyRelativeHumidity',
        'HourlyAltimeterSetting',
        'HourlyVisibility',
        'HourlyStationPressure',
        'HourlyWetBulbTemperature',
        'HourlyPrecipitation',
        'HourlyCloudCoverage',
        'HourlyCloudElevation',
        'HourlyWindSpeed'                   # weather end

    ],
    outputCol="features"
)

# Model

In [0]:
# baseline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from xgboost.spark import SparkXGBRegressor

from mlflow.models import infer_signature

mlflow.spark.autolog()
with mlflow.start_run(run_name="XGB-1y_baseline_all_feat"):
    MODEL_NAME = "XGB_1y_BASELINE_FEAT_ENG"

    # linear_reg = LinearRegression(
    #     featuresCol="features",
    #     labelCol="DEP_DELAY_NEW",
    #     # Linear Regression has different parameters than Random Forest
    #     maxIter=10, 
    #     regParam=0.3
    # )
    # rf = RandomForestRegressor(
    #     featuresCol="features",  
    #     labelCol="DEP_DELAY_NEW",   
    #     numTrees=20,
    #     maxDepth=10
    # )

    xgb_regressor = SparkXGBRegressor(
        features_col="features",
        label_col="DEP_DELAY_NEW",
        num_workers=2,
        max_depth=6,
        n_estimators=100,
        learning_rate=0.3
    )

    # Create pipeline
    pipeline = Pipeline(stages=[
        carrier_indexer, origin_indexer, dest_indexer, 
        carrier_encoder, origin_encoder, dest_encoder, 
        assembler,
        # linear_reg
        # rf
        xgb_regressor
    ])

    model = pipeline.fit(train_df)
    training_predictions = model.transform(train_df)
    validation_predictions = model.transform(validation_df)

    mae_evaluator = RegressionEvaluator(
        labelCol="DEP_DELAY_NEW",      
        predictionCol="prediction", 
        metricName="mae"           
    )

    rmse_evaluator = RegressionEvaluator(
        labelCol="DEP_DELAY_NEW",      
        predictionCol="prediction", 
        metricName="rmse"
    )

    # Calculate MAE
    mae_t = mae_evaluator.evaluate(training_predictions)
    mae_v = mae_evaluator.evaluate(validation_predictions)
    # Calculate RMSE
    rmse_t = rmse_evaluator.evaluate(training_predictions)
    rmse_v = rmse_evaluator.evaluate(validation_predictions)

    signature = infer_signature(train_df, training_predictions)

    mlflow.spark.log_model(
        model, 
        MODEL_NAME,
        input_example=train_df.limit(1).toPandas(),
        signature=signature,
        registered_model_name="flight_delay_prediction_baseline"
        )

    mlflow.log_metric("train_mae", mae_t)
    mlflow.log_metric("validation_mae", mae_v)
    mlflow.log_metric("train_rmse", rmse_t)
    mlflow.log_metric("validation_rmse", rmse_v)


## Save best model results

In [0]:
# best model information
RUN_ID = "100863169f14462fb514efa6483a170e"
ARTIFACT_PATH = "XGB_1y_BASELINE_FEAT_ENG"

MODEL_URI = f"runs:/{RUN_ID}/{ARTIFACT_PATH}"
# Load the model
loaded_model = mlflow.spark.load_model(MODEL_URI)